In [1]:
import pandas as pd
import numpy as np
import nltk




In [2]:

# create series of true and false. True is assigned for yes/no. False for undefine
def Extract (path):
    fd= open(path,encoding="utf-8", errors='replace')
    df = pd.read_csv(fd)   
    defined=df['class'] != ("undefined")
    # #output dataframe without undeined
    df2=df[defined]
    defined1=df2['class'] !="Undefined"
    df4=df2[defined1]
    #replace no PI with no
    df3=df4.replace("No PI","no")
    #replace PI with yes
    final=df3.replace("PI","yes")
    
    replace_yes=final.replace("Yes","yes")
    final_df= replace_yes.replace("No","no")
    return final_df, df


path='E:/DATA/Sem8/fyp/Training.csv'
final_df,df=Extract(path)
# print(df['class'].value_counts())
print(final_df)

     class                                               text
0       no  i'll  buy  the  iphone   x  if it can  get  ac...
1       no  Some dude in FB selling the  iPhone   X  64 gb...
2      yes  Home dab emote man today and I  get  hopped on...
3      yes                               Buy  an  iPhone   X 
4       no  I hate iOS 11. My  iPhone  6+ works 10X slower...
7       no  Bo-go sale tomorrow at T-Mobile  buy  one  get...
8      yes  I can  get  the  iPhone   X  here.... If I'm w...
9      yes  When you  buy  an  Iphone   X  with Unlimited ...
10     yes  @ Mrwhosetheboss  i really  wish  i  get  to m...
11      no  @ sprint  @sprintcare  if I switch to @TMobile...
12      no  Oneplus 5t has faster face recognition than  i...
13      no  Don't  buy  an  iPhone   X  if you're ugly. I ...
14      no  Could  buy  2x 5T with the price I paid for th...
15     yes  My friend shopping:  iPhone   X  is not that e...
16     yes  What if there was upgrade to an  iPhone  8 or ...
19      

In [3]:
# create series of true and false. False is assigned for yes/no. True is for undefined
undefined=df['class'] == "undefined" 
#output dataframe without undeined
df_undefine=df[undefined]
print(df_undefine)



          class                                               text
5     undefined  Brilliant @johnlewisretail - added  iPhone   X...
6     undefined  They doing that on purpose cause mine is to. T...
17    undefined                                So if I  buy  a  X 
18    undefined  Buy   iPhone   X  and  get  in the same and yo...
21    undefined  If @Spacekatgal  didn't  get  an  iPhone   X  yet
25    undefined  I ordered my pixel 2 on Tuesday from Google & ...
26    undefined  Would it be the same if I  buy  an  iPhone   X...
28    undefined  I wanna  buy  my mom a new  iPhone  so bad. I ...
31    undefined  @ AppleSupport   I have received the  iphone  ...
38    undefined  I  wish  I had a girl that rode for me  like  ...
41    undefined  You look  like  you stalk the uptown event the...
42    undefined  What you got  like  4  IPhone   X 's  buy  me ...
46    undefined  Who  need   iphone   x  I got the direct plug ...
51    undefined  Pre-registered customers at T World who will 

In [4]:
import re
from nltk.corpus import stopwords 
from nltk.tokenize import word_tokenize 


def read_stopwords(path):
    file1 = open(path,"r") 
    stopword = file1.readlines() 
    file1.close() 
    li_stopwords = stopword[0].split()
    return li_stopwords


def removeStopWords(df_punc_remove):
    # stop_words = set(stopwords.words('english')) 
    li_stopwords = read_stopwords("E:/DATA/Sem8/fyp/stopwords.txt")   
    # print(stop_words)
    count_clean = 0
    for text in df_punc_remove['text']:
        word_tokens = word_tokenize(text) 
        clean_text = ""
        for w in word_tokens: 
            if w.lower() not in li_stopwords: 
                if w.lower() == "iphone":
                    clean_text = clean_text + w.lower()+'x'+' '
                elif w.lower() == 'x':
                    clean_text = clean_text
                else: 
                    clean_text = clean_text + w.lower()+' ' 
        df_punc_remove.at[count_clean,'text'] = clean_text 
        count_clean += 1
    #return list of corpus without stop words in a list.        
    # print(df_punc_remove) 
    return df_punc_remove

def removePunc():
    count=0
    punc_remove_df=pd.DataFrame()
    for text in final_df['text']: 
        punc_remove_df.at[count,'text'] = re.sub(r'[^\w\s]','',text)
        punc_remove_df.at[count,'class'] = final_df.iloc[count]['class']
        count +=1
    #return corpus with out punctuation.
    # print(punc_remove_df)
    return punc_remove_df



def clean_data():
    df_punc_remove = removePunc()
    df_clean = removeStopWords(df_punc_remove)
    return df_clean


df_cleaned_text = clean_data()
# bprint(li_cleaned_text)

In [5]:
def text_concat(concat_df):
    text=""
    for x in concat_df["text"]:
        text=text +str(x)
    return text


text_corpus = text_concat(df_cleaned_text)



In [6]:
def doc_vector(unique, df_clean):
    data = np.zeros([df_clean['class'].count(), len(unique)])
    docVector = pd.DataFrame(data,columns=unique)
    docVector = docVector.assign(PurchaseIntention = list(df_clean['class']))
    # docVector['Purchase Intention'] = final_df['class']
    # print(docVector['PurchaseIntention'])
    doc_count = 0
    for doc in df_clean['text']:
        words = doc.split()
        for word in words:
            temp=word.lower()
            if temp in docVector.columns:
                docVector.at[doc_count, temp] += 1
        doc_count += 1
        
    return docVector


# docVector = DocVector() 

In [7]:
import math


def tf_idf(corpus):
    unique = list(set(corpus.split()))
    # print(unique)
    # tfIdf_df = pd.DataFrame(columns=unique)     
    tf_df = doc_vector(unique, df_cleaned_text)
    # idf_df = pd.DataFrame()
    total_docs = len(tf_df.index)
    for column in unique:
        num_doc_word = 0
        for no_doc in range(total_docs):
            if tf_df.at[no_doc,column] != 0:
                num_doc_word += 1
        idf = math.log(total_docs/num_doc_word)
        # idf_df.at[0,column] = idf
        tf_df[column] = tf_df[column].multiply(idf)
        idf = 0
    # print(tf_df['buy'])
    return tf_df
    # for col in unique:
    #     for tf_df[col]
    # 
                

df_tfIdf = tf_idf(text_corpus)    

In [8]:
# print(df_tfIdf.iloc[0])
# def featureExtract():
#     total_docs = len(df_tfIdf.index)
#     df_useful = pd.DataFrame()
#     for doc_id in range(total_docs):
#         doc_text = ""
#         for col in df_tfIdf.columns:
#             if col != 'PurchaseIntention':
#                 # print(type(df_tfIdf.at[doc_id,col]))
#                 if df_tfIdf.at[doc_id,col] > 0:
#                     # df_useful.at[doc_id,'text'] = df_useful.at[doc_id,'text'] + col + ' '
#                     doc_text = doc_text + col + ' '
#         df_useful.at[doc_id,'class'] = df_tfIdf.at[doc_id,'PurchaseIntention']  
#         df_useful.at[doc_id,'text'] = doc_text
#     print(df_useful) 
#     return df_useful
#     
#  
#     
# df_useful = featureExtract()   

In [9]:

# def make_unique_li(li_cleanText):
#     unique_words_set= set(li_cleanText)
#     unique_word_li = list(unique_words_set)
#     return unique_word_li
# 
# 
# def stemmed(li_cleanText):
#     count_stemed=0
#     for word in li_cleanText:
#         if word[-1]=="s":
#             li_cleanText[count_stemed]=word[:-1]
#         elif word[-2:]=="ed":
#             li_cleanText[count_stemed]=word[:-2]
#         elif word[-3:]=="ing":
#             li_cleanText[count_stemed]=word[:-3]
#         count_stemed+=1  
#     return li_cleanText   
# # li_stemmed = stemmed(li_cleaned_text)
# uniqueWords = make_unique_li(li_cleaned_text)
# print(uniqueWords)

uniqueWords = list(set(text_corpus.split()))
# docVector = doc_vector(uniqueWords, df_useful)
docVector = df_tfIdf

In [10]:
def WordGivenNoPI(tempNegDocVector):
    data = np.zeros([1,len(uniqueWords)])
    wordGivenNoPI = pd.DataFrame(data,columns=uniqueWords)
    columnSum = tempNegDocVector.sum(axis=1, skipna=True)
    numWordsInNoPI = columnSum.sum()
    
    for word in uniqueWords:
        nk_wordinNoPI = tempNegDocVector[word].sum()
        wordGivenNoPI.at[0,word] = (nk_wordinNoPI+1) / (numWordsInNoPI+len(uniqueWords))
    return wordGivenNoPI , numWordsInNoPI
    
    
def TrainModel():
    yesCount = docVector["PurchaseIntention"] == "yes"
    tempPosDocVector = docVector[yesCount]
    totalPI = tempPosDocVector["PurchaseIntention"].count()
    print("total PI ",totalPI)
    
    noCount = docVector["PurchaseIntention"] == "no"
    tempNegDocVector = docVector[noCount]
    # print(tempNegDocVector["PurchaseIntention"])
    totalNonPI = tempNegDocVector["PurchaseIntention"].count()
    print("total non PI ",totalNonPI)
    # print(totalPI+totalNonPI)
    # totalNonPI = docVector["PurchaseIntention"].count() - totalPI
    total = totalPI+totalNonPI
    Prob_PI = totalPI/total
    Prob_NoPI = totalNonPI/total
    
    data = np.zeros([1,len(uniqueWords)])
    wordGivenPI = pd.DataFrame(data,columns=uniqueWords)
    columnSum = tempPosDocVector.sum(axis=1, skipna=True)
    numWordsInPI = columnSum.sum()
    
    for word in uniqueWords:
        nk_wordinPI = tempPosDocVector[word].sum()
        wordGivenPI.at[0,word] = (nk_wordinPI+1) / (numWordsInPI+len(uniqueWords))
        
    df_wordGivenNoPI , numWordsInNoPI = WordGivenNoPI(tempNegDocVector) 
    return (wordGivenPI,df_wordGivenNoPI,Prob_PI,Prob_NoPI,numWordsInPI,numWordsInNoPI)
   

df_WordGivenPI,df_WordGivenNoPi,Prob_PI,Prob_NoPI,numWordsInPI,numWordsInNoPI = TrainModel()
print("prob pos ",Prob_PI)
print("prob_neg ",Prob_NoPI)

total PI  481
total non PI  274


prob pos  0.6370860927152318
prob_neg  0.3629139072847682


In [11]:
def Predict():
    test_path="E:/DATA/Sem8/fyp/test data/Testing.csv"
    test_data,test_df = Extract(test_path)
    # test_data = test_data.assign(PredictedClass= list(test_data['text']))
    # test_data = test_data[['class', 'PredictedClass', 'text']]
    # print(test_data["text"].count())
    predict_df = pd.DataFrame()
    weighPI = Prob_PI
    weighNoPI = Prob_NoPI
    count_test = 0
    for sentence in test_data['text']:
        # print(count_test)
        for word in sentence.lower().split():
            if word in uniqueWords:
                weighPI = weighPI * df_WordGivenPI.at[0,word]
                weighNoPI = weighNoPI * df_WordGivenNoPi.at[0,word]
            else:
                 weighPI = weighPI * (1/(numWordsInPI+len(uniqueWords)))
                 weighNoPI = weighNoPI * (1/(numWordsInNoPI+len(uniqueWords)))
        predict_df.at[count_test,'WeightPI'] = weighPI    
        if weighPI > weighNoPI:
            predict_df.at[count_test,'PredictedClass'] = 'yes'
            # print(test_data.at[count_test,'text'],test_data.at[count_test,'PredictedClass'])
        else:
           predict_df.at[count_test,'PredictedClass'] = 'no'
           # print(test_data.at[count_test,'text'],test_data.at[count_test,'PredictedClass'])
            
        count_test += 1
        weighPI = Prob_PI
        weighNoPI = Prob_NoPI  
    print(test_data['class'], predict_df['PredictedClass'])    
    return predict_df, test_data  
    
    
    # print("PI weight {0} No PI weight{1}".format(weighPI,weighNoPI))        


predict_df, test_data = Predict()        



0      yes
2      yes
3      yes
5      yes
6      yes
8      yes
15     yes
16      no
21     yes
23     yes
24      no
26     yes
27     yes
31     yes
32     yes
38     yes
39     yes
40     yes
41     yes
44     yes
49     yes
57      no
60     yes
63     yes
65     yes
66     yes
67     yes
69     yes
70     yes
76     yes
      ... 
462    yes
465    yes
466    yes
467    yes
468    yes
469    yes
473    yes
474    yes
475    yes
481    yes
482    yes
483    yes
486     no
487     no
488    yes
490    yes
492    yes
493    yes
498    yes
499    yes
500    yes
501    yes
502    yes
503    yes
504    yes
505    yes
511    yes
512    yes
514    yes
515     no
Name: class, Length: 253, dtype: object 0       no
1      yes
2      yes
3       no
4      yes
5      yes
6      yes
7       no
8      yes
9       no
10     yes
11     yes
12      no
13      no
14     yes
15      no
16     yes
17      no
18      no
19     yes
20      no
21      no
22     yes
23     yes
24     yes
25      no
26 

In [12]:
def ConfusionMatrix():
    TP=0
    TN=0
    FP=0
    FN=0
    for i in range(test_data['class'].count()):
        # print(test_data.iloc[i]['class']," ",predict_df.iloc[i]['PredictedClass'] )
        if test_data.iloc[i]['class'] == "yes" and predict_df.iloc[i]['PredictedClass'] == "yes":
            TP += 1
        elif test_data.iloc[i]['class'] == "yes" and predict_df.iloc[i]['PredictedClass']=="no":
            FN += 1
        elif test_data.iloc[i]['class'] == "no" and predict_df.iloc[i]['PredictedClass']=="no":
            TN += 1
        elif test_data.iloc[i]['class'] == "no" and predict_df.iloc[i]['PredictedClass'] == "yes":
            FP += 1    
    return TP,FN,TN,FP
 
 
TP,FN,TN,FP = ConfusionMatrix()
print("TP {0} , FN {1}, TN {2}, FP {3}".format(TP,FN,TN,FP))        

TP 83 , FN 120, TN 38, FP 12


In [ ]:
def Accuracy():
    accuracy=(TP+TN)/(TP+FP+FN+TN)
    # print("Accuracy =",accuracy)
    return accuracy
   
def Precision():
    precision =  TP/(TP+FP)
    # print("Precision = ",precision)
    return precision

def Recall():
    recall = TP/(TP+FN)
    # print("Recall = ",recall)
    return recall
    
def fScore():
    F1 = 2*(Recall() * Precision()) / (Recall() + Precision())
    # print("f measure",F1)
    return F1


print("Accuracy = ",Accuracy())
print("Precision = ",Precision())
print("Recall = ",Recall())
print("fScore = ",fScore())

Accuracy =  0.4505928853754941
Precision =  0.872093023255814
Recall =  0.3694581280788177
fScore =  0.5190311418685121


In [ ]:


# # import numpy as np
# from sklearn.metrics import roc_curve
# import matplotlib.pyplot as plt
# import matplotlib.pyplot as plt
# y = test_data['class']
# scores = predict_df['WeightPI']
# fpr, tpr, thresholds = roc_curve(y, scores, pos_label="yes")
# 
# # print(thresholds)
# plt.plot(fpr, tpr)
# 
# plt.show()

# docVector= docVector.values
# print(type(docVector))
# print(docVector)
